In [1]:
import keras
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg16 import preprocess_input
import matplotlib as plt

Using TensorFlow backend.


In [2]:
from volume_estimator1 import VolumeEstimator 
import sys
import json
from keras.models import Model, model_from_json
from food_volume_estimation.depth_estimation.custom_modules import *
from food_volume_estimation.food_segmentation.food_segmentator import FoodSegmentator
import matplotlib.pyplot as plt
from pyntcloud import PyntCloud

# Paths to model archiecture/weights
depth_model_architecture = 'monovideo_fine_tune_food_videos.json'
depth_model_weights = 'monovideo_fine_tune_food_videos.h5'
segmentation_model_weights = 'mask_rcnn_food_segmentation.h5'

# Create estimator object and intialize
estimator = VolumeEstimator(arg_init=False)
with open(depth_model_architecture, 'r') as read_file:
    custom_losses = Losses()
    objs = {'ProjectionLayer': ProjectionLayer,
            'ReflectionPadding2D': ReflectionPadding2D,
            'InverseDepthNormalization': InverseDepthNormalization,
            'AugmentationLayer': AugmentationLayer,
            'compute_source_loss': custom_losses.compute_source_loss}
    model_architecture_json = json.load(read_file)
    estimator.monovideo = model_from_json(model_architecture_json, custom_objects=objs)
estimator._VolumeEstimator__set_weights_trainable(estimator.monovideo, False)
estimator.monovideo.load_weights(depth_model_weights)
estimator.model_input_shape = estimator.monovideo.inputs[0].shape.as_list()[1:]
depth_net = estimator.monovideo.get_layer('depth_net')
estimator.depth_model = Model(inputs=depth_net.inputs, outputs=depth_net.outputs, name='depth_model')
print('[*] Loaded depth estimation model.')

E:\Anaconda3\envs\caps_food\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[*] VolumeEstimator not initialized.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use tf.cast instead.
[*] Loaded depth estimation model.


In [3]:
# Depth model configuration
MIN_DEPTH = 0.01
MAX_DEPTH = 10
estimator.min_disp = 1 / MAX_DEPTH
estimator.max_disp = 1 / MIN_DEPTH
estimator.gt_depth_scale = 0.333 # Ground truth expected median depth

In [4]:
# Create segmentator object
estimator.segmentator = FoodSegmentator(segmentation_model_weights)

# Set plate adjustment relaxation parameter
estimator.relax_param = 0.01

[*] Loading segmentation model weights mask_rcnn_food_segmentation.h5


In [5]:
model_classifier = load_model('model.h5')

Instructions for updating:
Use tf.cast instead.


In [6]:
import cv2
import numpy as np
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

# Define configuration for the model
class InferenceConfig(Config):
    # Give the configuration a recognizable name
    NAME = "segmentation_of_food"

    # Set batch size to 1 since we're running inference on a single image
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Number of classes (including background)
    NUM_CLASSES = 2  # 1 Background + 1 Object

# Create config object
config = InferenceConfig()

# Create Mask R-CNN model in inference mode
model_seg = MaskRCNN(mode="inference", config=config, model_dir="./logs")

# Load pre-trained weights
model_seg.load_weights(segmentation_model_weights, by_name=True)
clusters = ['food']
class_names = ['bg'] + clusters

In [7]:
def predict(path = None,field_of_view= 70,dia = 0,plot_results=False,food = None):


    # Estimate volumes in input image
    input_image = path
    plate_diameter = dia # Set as 0 to ignore plate detection and scaling(diameter is in meter)
    if food==None:

#         # Load the image using the PIL library
#         image = Image.open(path)

#         # Check if height is greater than width
#         if image.height > image.width:
#             # Convert the image to a NumPy array
#             image_array = np.array(image)

#             # Transpose the image array
#             transposed_array = np.transpose(image_array, (1, 0, 2))

#             # Convert the transposed array back to an image
#             transposed_image = Image.fromarray(transposed_array)
#             image=transposed_image
#             # Display the transposed image
#             #transposed_image.show()
#         else:
#             # Display the original image
#             image=image
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = model_seg.detect([image], verbose=0)
        r = results[0]
        out_put = []
        for i in range(r["rois"].shape[0]):
            masked_image = cv2.bitwise_and(image, image, mask=r["masks"][:,:,i].astype(np.uint8))
            masked_image = cv2.resize(masked_image,(224,224))
            out_put.append(masked_image)
        lable = []
        for i in out_put:
            img_array = np.expand_dims(i, axis=0)
            img_array = preprocess_input(img_array)
            result  = model_classifier.predict(img_array)
            classes =  ['alloo_matar', 'apple', 'banana', 'bhindi', 'carrot', 'cucumber', 'dal makhni', 'fried rice', 'Jalebi', 'orange', 'pizza', 'roti', 'samosa']
            res = np.argmax(result)
            lable.append(classes[res])

        outputs_list = estimator.estimate_volume(input_image, fov=field_of_view, plate_diameter_prior=plate_diameter, 
                                                 plot_results=plot_results,lable=lable)
        print(lable)
    else: 
        outputs_list = estimator.estimate_volume(input_image, fov=field_of_view, plate_diameter_prior=plate_diameter, 
                                                 plot_results=plot_results,lable=food)
        lable=[food]
    
    if plot_results:
        out = []
        for i in range(len(outputs_list)):
            out.append(outputs_list[i][0])
    else:
        out = outputs_list
        
    Calories = {}
    weights = {}
    calaroies = pd.read_csv("Calories.csv")
    if len(lable)==len(out):
        for i in range(len(out)):
            vol = out[i]*1000000
            ind = list(calaroies.loc[calaroies.name==lable[i]].index)[0]
            wei = calaroies.density[ind] * vol
            calory = (wei/100)*calaroies.cal[ind]
            if wei>10:
                if lable[i] in Calories.keys(): 
                    Calories[lable[i]] += calory
                    weights[lable[i]] += wei
                else:
                    Calories[lable[i]] = calory
                    weights[lable[i]] = wei
    else:
        for i in range(len(out)):
            vol = out[i]*1000000
            ind = list(calaroies.loc[calaroies.name==lable[0]].index)[0]
            wei = calaroies.density[ind] * vol
            calory = (wei/100)*calaroies.cal[ind]
            if lable[0] in Calories.keys(): 
                Calories[lable[0]] += calory
                weights[lable[0]] += wei
            else:
                Calories[lable[0]] = calory
                weights[lable[0]] = wei
                
    total_cal = np.sum(list(Calories.values()))
    total_weight = np.sum(list(weights.values()))
    print("total Calories your are goint to intake is",round(total_cal,2),"cal")
    print("total weight of food is",round(total_weight,2),"gms")
    
    return Calories,weights
    

In [8]:
predict('carrot.jpg',plot_results=False,food="carrot")

Found 1 food object(s) in image.
total Calories your are goint to intake is 301.85 cal
total weight of food is 736.59 gms


({'carrot': 301.8546071433863}, {'carrot': 736.5900613552619})